# Andersen, Vinther & Ottesen (2013) Model Code <a name="top" />

# Table of Contents
1. [Instructions](#instructions)
    1. [Parameter Optimization Against TSST Data Sets](#TSSTInstructions)
    2. [Parameter Optimization Against Basal Data Sets](#basalInstructions)
    3. [Running Without Parameter Optimization](#noOptInstructions)
2. [Imports](#imports)
3. [Parameters and Initial Conditions](#params)
4. [Put Raw Data into Arrays](#rawdata)
    1. [Plot Data Sets](#plotdata)
5. [Model Function--Includes ODE Solver](#modelfunction)
6. [Cost Function Definition](#cost)
7. [Run the Optimization](#run)
8. [Save Output to File](#saveoutput)
9. [Compute Means and Std Devations of Parameters and Output as Table](#paramtable)
10. [Plots](#plots)
11. [No Optimization Run and Plot](#no-opt)
12. [Reproducing Figure 6.1 from Paper](#reproducing)
13. [Run from Opt_pars File](#run-optpars)
14. [Dependencies](#dependencies)

## Instructions <a name="instructions" />

### Parameter Optimization Against TSST Data Sets <a name="TSSTInstructions" />

**Note:** To quickly run a cell (or a selection of cells), use the shortcut Shift+Enter (or you can also use the button labeled "Run" in the toolbar at the top).

To run simulations with parameter optimization against TSST data, there is no need to change any cells until the cost function cell. Simply run all cells up to the cell below the heading **Cost Function Definition**.

In order to set which data set to optimize parameters against, look for the following line of code in the cost function definition:
    
    return costFun.SSE_cost(...)

**Note:** You also have the option of using a cost function based on the maximum distance between simulation and real-world data. Simply change SSE_cost to MAX_cost, the instructions for function arguments remain the same.

In order to run against a patient from the TSST data sets, simply change the second and fourth arguments to reflect the patient number and subject group. The subject groups are:

- nelson.melancholicACTH & nelson.melancholicCortisol (15 patients)
- nelson.atypicalACTH & nelson.atypicalCortisol (14 patients)
- nelson.neitherACTH & nelson.neitherCortisol (14 patients)
- nelson.healthyACTH & nelson.healthyCortisol (15 patients)

You could also run against the mean of all patients cortisol and ACTH concentration values by using `nelson.ACTH[:,1]` and `nelson.cortisol[:,1]`. Or you can run against the mean of any subgroup using `nelson.<subgroup name>Cortisol_mean[:,1]` and `nelson.<subgroup name>ACTH_mean[:,1]` (for instance `nelson.melancholicCortisol_mean[:,1]` & `nelson.melancholicACTH_mean[:,1]`). 

Note that the first column in each data set is the time steps, so indexing with `[:,0]` is referring to the time. These are the values we need to pass as the first (ACTH time steps) and third (cortisol time steps) arguments to the cost function.

The following are several examples of arguments you could pass the cost function with explanations:

- `nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1], simData`
    - The 1st patient in the Melancholic subgroup
- `nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14], simData`
    - The 14th patient in the Atypical subgroup
- `nelson.healthyACTH[:,0], nelson.healthyACTH[:,2], nelson.healthyCortisol[:,0], nelson.healthyCortisol[:,2], simData`
    - The 2nd patient in the Healthy Control group
- `nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1], simData`
    - The mean data set for all patients (depressed and control)
- `nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1], simData`
    - The mean of all control patients
    
In order to find out the actual Patient ID of the patient you are matching, try creating a new cell and entering the following command (using the subtype name and index number you used in the cost function arguments):

    print(melancholic_ids[1])
    
Next, you need to set the initial conditions for each ODE. In the cell directly below the heading **Run the Optimization**, you'll need to change the following line:

    y0 = [1, nelson.ACTH[0,1], nelson.cortisol[0,1]]
    
Depending on which data set you used, change `nelson.ACTH` and `nelson.cortisol` to match. Then, depending on which patient you used, change `[0,1]` in both places to `[0,#]` where # is replaced with the number from above. 

Alternatively, if you used the mean of all patients, you'd leave it unchanged.
    
And for each of the subgroup means, you'd use (or simply replace Healthy with whichever other subgroup you used):

    y0 = [1, nelson.healthyACTH_mean[0,1], nelson.healthyCortisol_mean[0,1]]
    
At this point, you are ready to run the optimization, so simply run the cells up to the heading **Save Output to File**. This may take some time, so while it is running you can move on to the next steps (if you run a cell while another is processing, it will add it to a queue).

The next few cells under the heading **Save Output to File** can be changed so that the filenames match the subgroup and patient you are running. For instance, the filename below is for the Melancholic subgroup patient with ID 3:

    df_t.to_excel(excel_writer = 'andersenModel_output/andersenModel-nelson-melancholic-patientID_3-timeSteps-5-
        iterations.xlsx')
    
The final step after saving the outputs is to plot the simulations against the real-world data. The cell under the heading **Plots** contains the code for this purpose. The lines of concern to ensure that you're showing the data set that you matched are (marked with comments in the actual code cell):
```
ax2.plot(nelson.ACTH[:,0], nelson.ACTH[:,1], label = "Nelson ACTH Data - Patient Mean", color = "orange")
ax3.plot(nelson.cortisol[:,0], nelson.cortisol[:,1], label = "Nelson Cortisol Data - Patient Mean", 
    color = "orange")
```
Here, change the first two arguments of each line to match exactly the arguments you used for the cost function. The labels can be changed to the patient subgroup and Patient ID you matched, also. And the filename for the figure can be changed in the last line of the cell:

    plt.savefig("andersenModel_output/andersenModel-nelson-melancholic-patientID_3-5-iterations-all-params-
        normalizedCost.png", dpi = 300)
    
### Parameter Optimization Against Basal Data Sets <a name="basalInstructions" />

Since these data sets have data points over a 24-hour period (1440 minutes), rather than 140.01 minutes, you will need to change the time interval over which the ODE solver integrates. To do this, go to the cell directly above the heading **Put Raw Data Into Arrays** and uncomment (delete the # at the start of the line) the lines:

    t_start = -0.01
    t_end = 1455.01
    t_step = 0.01

You'll need to comment out the other definitons for these variables (place a # at the start of the line).

The reason you add the extra 15 minutes is that you need to make sure that when you interpolate between your simulated data points the line covers every real-world data point so that you don't cause issues when computing the cost function (and the last data point for the Golier cortisol concentration data sets is at 1455 minutes).

After making this change, you need to again change the cost function arguments so that you are matching the basal data set in which you are interested. This time, however, you will also need to change the first and third arguments, because we need to tell the function the correct time steps for the data set.

First, choose which data set you wish to match. Here are the options:

- yehuda.controlCortisol
- yehuda.PTSDCortisol
- yehuda.depressedCortisol
- carroll.controlCortisol & carroll.controlACTH
- carroll.LCDepressedCortisol & carroll.LCDepressedACTH (LC = Low Cortisol)
- carroll.HCDepressedCortisol & carroll.HCDepressedACTH (HC = High Cortisol)
- golier.PTSDCortisol & golierPTSDACTH
- golier.nonPTSDTraumaExposedCortisol & golier.nonPTSDTraumaExposedACTH
- golier.nonPTSDNonExposedCortisol & golier.nonPTSDNonExposedACTH
- bremner.abusedPTSDCortisol
- bremner.nonAbusedPTSDCortisol
- bremner.nonAbusedNonPTSDCortisol

**Note:** To see what any of these data sets looks like, click on the **Plot Basal Data Sets** heading in the Table of Contents.

**Note Also:** These data sets all come in smoothed versions (each data point is set to the average of the nearest 5 points of the unsmoothed data). Also, the data sets by Carroll, Golier and Bremner also come in rearranged (or smoothed & rearranged) versions to match the starting time of the Yehuda data (10AM). To use any of these versions, simply append one of the following tags to the end of the data set name (before the indices): `_smooth`, `_rearr`, or `_rearr_smooth`.

First, I will cover what to do with data sets that contain both ACTH and cortisol values, and then afterwards I will cover using the Yehuda and Bremner data sets (which have only cortisol concentration data). For all of these data sets, the first column is the time step values. This means that if you take any of these arrays and index it with `[:,0]`, you are referring to the time steps. These are the values we need to pass as the first (ACTH time steps) and third (cortisol time steps) arguments to the cost function.

Then for the second and fourth arguments, you index the same data sets with `[:,1]` to mean the second column (which contains the mean concentration values for each patient group).

Here are a couple of examples showing arguments you can pass to the cost function:

- `carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1], simData`
    - The smoothed Control group mean for the Carroll data set
- `golier.nonPTSDTraumaExposedACTH[:,0], golier.nonPTSDTraumaExposedACTH[:,1], golier.nonPTSDTraumaExposedCortisol[:,0], golier.nonPTSDTraumaExposedCortisol[:,1], simData`
    - The Trauma-Exposed Control group mean for the Golier data set
    
In order to run simulations against data sets that do not include ACTH concentration data, you will need to change the name of the function to `costFun.SSE_cost_noACTH` and then simply do not include the two arguments for ACTH data. To use the Yehuda Control group data, this would look like:

    return costFun.SSE_cost_noACTH(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], simData)

You'll also need to change the initial conditions in the cell below the heading **Run the Optimization**, again. Use the name of the data set you used for the cost function, and change the indices to `[0,1]` for both. Here's an example (using the smoothed Carroll Control group data set):

    y0 = [1, carroll.controlACTH_smooth[0,1], carroll.controlCortisol_smooth[0,1]]
    
Next, because you changed the length of the interval over which the ODE solver integrates, you'll need to change the arrays initialized in the next cell to be longer. For each of the last 4 lines in the cell, you'll need to replace the number 14002 with 145502 (since we start at -0.01 and go until 1455.01 with a step size of 0.01, that's 145502 steps total).

At this point, you're ready to run the parameter optimization, so run the next cell. Again, it may take a while, so you can start editing the remaining cells while you wait.

The cells under the heading **Save Output to File** should each have the filename changed to something that reflects the data set you're matching now. For instance, the Excel filename for sims_acth when matching the smoothed Carroll Control group would become:

    df_crh.to_excel(excel_writer = 'andersenModel_output/andersenModel-carroll-control-smooth-sims-crh-5-
        iterations.xlsx')
            
Finally, the cell under the heading **Plots** needs to have the same lines changed as described above. For instance, when matching the smoothed Carroll Control group, they would become:
```
ax2.plot(carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], label = "Carroll Control ACTH Data", 
    color = "orange")
ax3.plot(carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1], label = "Carroll Control 
    Cortisol Data", color = "orange")
```
The data set names and indices here should match exactly the names and indices used as arguments to the cost function. Also, the last line of the cell can have the filename changed as described above to reflect which data set you are plotting.

### Running Without Parameter Optimization <a name="noOptInstructions" />

To reproduce Figure 6.1 from the Andersen et al. (2013) paper, you don't need to edit any code. Simply run the cells in the section labeled **Reproducing Figure 6.1 from Paper**.

To run the model with any set of paramaters you desire, without optimization, you can use the cells under the heading **No Optimization Run and Plot**.

Set the parameters, initial conditions and time interval you want to use by changing the values defined in the section **Parameters and Initial Conditions**. You can then run the cells under the heading **No Optimization Run and Plot**.

[Back to Top](#top)

## Imports <a name="imports"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco
from scipy.interpolate import interp1d
import mpld3
from tabulate import tabulate
import matplotlib
import pandas as pd
from HPAmodeling import DEsolver, costFun
from HPAmodeling.dataImport import data

[Back to Top](#top)

## Parameters and Initial Conditions <a name="params"></a>

In [ ]:
# initial conditions
# order: CRH, ACTH, CORT

# based on: golierPTSD
y0 = [10, 44.22111, 10.68792]

In [ ]:
# set the values for time delays (30 and 60 are the values used in Bairagi et al. for the time delays)
#
# the authors did not include time delays in the paper, I'm simply trying this out to see if it yields
# oscillating solutions because I think that is likely to be the case
tau1 = 30
tau2 = 60

In [ ]:
# authors' listed parameter values
k0 = 0.859    # pg/ml*min
k1 = 0.127    # 1/min
k2 = 0.00132  # 1/min
w1 = 0.173    # 1/min
w2 = 0.0348   # 1/min
w3 = 0.00907  # 1/min
rho = 0.5
psi = 0.5
xi = 2
alpha = 3
gamma = 3
c = 3.06      # ng/ml
c3 = 1.42     # ng/ml

# save these parameter values into an array for running without parameter optimization
authors_params = [k0, k1, k2, w1, w2, w3, rho, psi, xi, alpha, gamma, c, c3]

In [ ]:
# compute bounds based on +- 10%
bound = c3
print(bound - bound*.1)
print(bound + bound*.1)

In [ ]:
# bounds for parameter optimization
# order is: k0, k1, k2, w1, w2, w3, rho, psi, xi, alpha, gamma, c, c3
# starting with +- 10% since we do not have published ranges in the paper
bounds = [(0.7731, 0.9449), (0.1143, 0.1397), (0.001188, 0.001452), (0.15569999999999998, 0.1903), (0.03132, 0.038279999999999995), (0.008163, 0.009977), (0.45, 0.55), (0.45, 0.55), (1.8, 2.2), (2.7, 3.3), (2.7, 3.3), (2.754, 3.366), (1.278, 1.5619999999999998)]

In [ ]:
# define time interval for integration

# time interval and step definition
# all data sets end on 1440.0 or earlier except the Golier cortisol sets,
# they end on 1455.0, so I should set t_end = 1455.01 when matching them
#t_start = -0.01
#t_end = 1455.01
#t_step = 0.01

# for matching Nelson data, use these values of t_start, t_end and t_step
t_start = -0.01
t_end = 140.01
t_step = 0.01

[Back to Top](#top)

## Put Raw Data into Arrays <a name="rawdata"></a>

In [ ]:
# Create an instance of the data class for each data set contained in the HPAmodeling library, and set the time
# scale to minutes.
yehuda = data("yehuda", "minutes")
carroll = data("carroll", "minutes")
golier = data("golier", "minutes")
bremner = data("bremner", "minutes")
nelson = data("nelson", "minutes")

### Plot Data Sets <a name="plotdata"></a>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], label = "Control Group Cortisol")
ax1.plot(yehuda.controlCortisol_smooth[:,0], yehuda.controlCortisol_smooth[:,1], label = "Control Group Cortisol - Smoothed")
ax1.set(xlabel="Time (minutes)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(yehuda.PTSDCortisol[:,0], yehuda.PTSDCortisol[:,1], label = "PTSD Group Cortisol")
ax2.plot(yehuda.PTSDCortisol_smooth[:,0], yehuda.PTSDCortisol_smooth[:,1], label = "PTSD Group Cortisol - Smoothed")
ax2.set(xlabel="Time (minutes)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(yehuda.depressedCortisol[:,0], yehuda.depressedCortisol[:,1], label = "Depression Group Cortisol")
ax3.plot(yehuda.depressedCortisol_smooth[:,0], yehuda.depressedCortisol_smooth[:,1], label = "Depression Group Cortisol - Smoothed")
ax3.set(xlabel="Time (minutes)", ylabel="Cortisol (micrograms/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
#mpld3.enable_notebook()
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4, figsize = (15,15))

ax1.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax1.plot(carroll.HCDepressedCortisol_rearr[:,0], carroll.HCDepressedCortisol_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax1.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax1.plot(carroll.HCDepressedCortisol_rearr_smooth[:,0], carroll.HCDepressedCortisol_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax1.set(xlabel="Time (minutes)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="upper right", shadow = True, fancybox = True)

ax2.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax2.plot(carroll.LCDepressedCortisol_rearr[:,0], carroll.LCDepressedCortisol_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax2.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax2.plot(carroll.LCDepressedCortisol_rearr_smooth[:,0], carroll.LCDepressedCortisol_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax2.set(xlabel="Time (minutes)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="upper right", shadow = True, fancybox = True)

ax3.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax3.plot(carroll.HCDepressedACTH_rearr[:,0], carroll.HCDepressedACTH_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax3.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax3.plot(carroll.HCDepressedACTH_rearr_smooth[:,0], carroll.HCDepressedACTH_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax3.set(xlabel="Time (minutes)", ylabel="ACTH (pg/mL)")
ax3.legend(loc="upper right", shadow = True, fancybox = True)

ax4.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax4.plot(carroll.LCDepressedACTH_rearr[:,0], carroll.LCDepressedACTH_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax4.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax4.plot(carroll.LCDepressedACTH_rearr_smooth[:,0], carroll.LCDepressedACTH_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax4.set(xlabel="Time (minutes)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="upper right", shadow = True, fancybox = True)

In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(nrows = 6, figsize = (15,20))

ax1.plot(golier.PTSDCortisol_rearr_smooth[:,0], golier.PTSDCortisol_rearr_smooth[:,1], label = "Trauma Exposed PTSD Cortisol - Smoothed")
ax1.plot(golier.PTSDCortisol_rearr[:,0], golier.PTSDCortisol_rearr[:,1], label = "Trauma Exposed PTSD Cortisol")
ax1.set(xlabel="Time (minutes)", ylabel="Cortisol (mg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,0], golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD Cortisol - Smoothed")
ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr[:,0], golier.nonPTSDTraumaExposedCortisol_rearr[:,1], label = "Trauma Exposed Non-PTSD Cortisol")
ax2.set(xlabel="Time (minutes)", ylabel="Cortisol (mg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(golier.nonPTSDNonExposedCortisol_rearr_smooth[:,0], golier.nonPTSDNonExposedCortisol_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD Cortisol - Smoothed")
ax3.plot(golier.nonPTSDNonExposedCortisol_rearr[:,0], golier.nonPTSDNonExposedCortisol_rearr[:,1], label = "Non-Exposed Non-PTSD Cortisol")
ax3.set(xlabel="Time (minutes)", ylabel="Cortisol (mg/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)

ax4.plot(golier.PTSDACTH_rearr_smooth[:,0], golier.PTSDACTH_rearr_smooth[:,1], label = "Trauma Exposed PTSD ACTH - Smoothed")
ax4.plot(golier.PTSDACTH_rearr[:,0], golier.PTSDACTH_rearr[:,1], label = "Trauma Exposed PTSD ACTH")
ax4.set(xlabel="Time (minutes)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="lower right", shadow = True, fancybox = True)

ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,0], golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD ACTH - Smoothed")
ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr[:,0], golier.nonPTSDTraumaExposedACTH_rearr[:,1], label = "Trauma Exposed Non-PTSD ACTH")
ax5.set(xlabel="Time (minutes)", ylabel="ACTH (pg/mL)")
ax5.legend(loc="lower right", shadow = True, fancybox = True)

ax6.plot(golier.nonPTSDNonExposedACTH_rearr_smooth[:,0], golier.nonPTSDNonExposedACTH_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD ACTH - Smoothed")
ax6.plot(golier.nonPTSDNonExposedACTH_rearr[:,0], golier.nonPTSDNonExposedACTH_rearr[:,1], label = "Non-Exposed Non-PTSD ACTH")
ax6.set(xlabel="Time (minutes)", ylabel="ACTH (pg/mL)")
ax6.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(bremner.abusedPTSDCortisol_rearr_smooth[:,0], bremner.abusedPTSDCortisol_rearr_smooth[:,1], label = "Abused PTSD Cortisol - Smoothed")
ax1.plot(bremner.abusedPTSDCortisol_rearr[:,0], bremner.abusedPTSDCortisol_rearr[:,1], label = "Abused PTSD Cortisol")
ax1.set(xlabel="Time (minutes)", ylabel="Cortisol (microg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(bremner.nonAbusedPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused PTSD Cortisol - Smoothed")
ax2.plot(bremner.nonAbusedPTSDCortisol_rearr[:,0], bremner.nonAbusedPTSDCortisol_rearr[:,1], label = "Non-Abused PTSD Cortisol")
ax2.set(xlabel="Time (minutes)", ylabel="Cortisol (microg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused Non-PTSD Cortisol - Smoothed")
ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr[:,0], bremner.nonAbusedNonPTSDCortisol_rearr[:,1], label = "Non-Abused Non-PTSD Cortisol")
ax3.set(xlabel="Time (minutes)", ylabel="Cortisol (microg/dL)")
ax3.legend(loc="lower left", shadow = True, fancybox = True)


In [ ]:
# graph the Nelson data (only showing one patient at a time currently)
fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15, 15))

ax1.plot(nelson.ACTH[:,0], nelson.ACTH[:,1])
ax2.plot(nelson.cortisol[:,0], nelson.cortisol[:,1])

[Back to Top](#top)

## Model Function--Includes ODE Solver <a name="modelfunction"></a>

In [ ]:
def model(params, ics):
    def ode_system(t, y):
        dy = np.zeros(3) # 3 equation system, x1 = CRH, x2 = ACTH, x3 = CORT
        
        [k0, k1, k2, w1, w2, w3, rho, psi, xi, alpha, gamma, c, c3] = params
        
        dy[0] = k0*(1 + xi*((DEsolver.delayedCORT**alpha)/(DEsolver.delayedCORT**alpha + c**alpha)) - psi*((DEsolver.delayedCORT**gamma)/(DEsolver.delayedCORT**gamma + c3**gamma))) - w1*y[0]
        dy[1] = k1*(1 - rho*((DEsolver.delayedCORT**alpha)/(DEsolver.delayedCORT**alpha + c**alpha)))*y[0] - w2*y[1]
        dy[2] = k2*DEsolver.delayedACTH - w3*y[2]
        
        return dy
    
    # Call the solve() function from my DEsolver module, and pass all of the information it needs.
    # Arguments are as follows: ODE function to solve, array of initial conditions, start time, step size, end time
    # The last three arguments are optional (leave blank for ODE systems) for delay differential equation systems, 
    #  tau1 is the delay in ACTH, tau2 is the delay in CORT, and delay is a boolean to set whether we solve with delays
    timeSeries = DEsolver.solve(ode_system, ics, t_start, t_step, t_end, tau1 = 15, tau2 = 30, delay = [False, True, True], delay_rough = False)
    return timeSeries

[Back to Top](#top)

## Cost Function Definition <a name="cost"></a>

In [ ]:
def cost_fun(params):
    simData = model(params, y0)
    
    return costFun.SSE_cost(nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1], simData)

[Back to Top](#top)

## Run the Optimization <a name="run"></a>

In [ ]:
# initial conditions
# order: CRH, ACTH, CORT

# based on: golierPTSD
y0 = [1, nelson.ACTH[0,1], nelson.cortisol[0,1]]

In [ ]:
# number of times to run the optimization
n = 1

# define an array to hold the population of parameter vectors
opt_pars = np.zeros((n, len(bounds)+1))

# initialize arrays to save simulation cortisol and ACTH data from each optimization
sims_cort = np.zeros((14002, n))
sims_acth = np.zeros((14002, n))
sims_crh = np.zeros((14002, n))


In [ ]:
%%time

# loop n times, running the optimization each time
for i in range(0,n):
    
    print(f"Optimization Run #{i+1}")
    
    # call the differential evolution optimization function on the cost function
    res = sco.differential_evolution(cost_fun, bounds, maxiter = 999, disp = True, popsize = 1)
    
    # alternatively, we can run the SHGO algorithm with the sampling_method = "sobol" flag to do global
    #     optimization with reporting all local minima, as well
    #res = sco.shgo(cost_fun, bounds, callback=callback_fun(*shgo_iter_steps), options = {"f_min": 0.1, "maxiter": None, "minimize_every_iter": True, "local_iter": False, "disp": True}, iters = 3)
    #res = sco.basinhopping(cost_fun, x0, niter = 1000)
    #res = sco.dual_annealing(cost_fun, bounds)
    
    # plug the optimized parameters into the solver
    optimizedSimData = model(res.x, y0)
    # save CRH, cortisol and ACTH data into sims arrays
    sims_cort[:,i] = optimizedSimData[:,3]
    sims_acth[:,i] = optimizedSimData[:,2]
    sims_crh[:,i] = optimizedSimData[:,1]
    
    # save the cost function values and optimized parameters for each iteration into the array opt_pars
    opt_pars[i,0] = res.fun
    opt_pars[i,1:] = res.x

[Back to Top](#top)

## Save Output to File <a name="saveoutput"></a>

In [ ]:
df_t = pd.DataFrame(optimizedSimData[:,0])
df_cort = pd.DataFrame(sims_cort)
df_acth = pd.DataFrame(sims_acth)
df_crh = pd.DataFrame(sims_crh)

df_t.to_excel(excel_writer = 'andersenModel_output/andersenModel-nelson-patientMean-timeSteps-5-iterations-normalizedCost.xlsx')
df_cort.to_excel(excel_writer = 'andersenModel_output/andersenModel-nelson-patientMean-sims-cort-5-iterations-normalizedCost.xlsx')
df_acth.to_excel(excel_writer = 'andersenModel_output/andersenModel-nelson-patientMean-sims-acth-5-iterations-normalizedCost.xlsx')
df_crh.to_excel(excel_writer = 'andersenModel_output/andersenModel-nelson-patientMean-sims-crh-5-iterations-normalizedCost.xlsx')

In [ ]:
np.savetxt('andersenModel_output/andersenModel-nelson-patientMean-opt-pars-5-iterations-normalizedCost.txt', opt_pars)

In [ ]:
np.savetxt('andersenModel_output/andersenModel-nelson-patientMean-sims-crh-5-iterations-normalizedCost.txt', sims_crh)
np.savetxt('andersenModel_output/andersenModel-nelson-patientMean-sims-acth-5-iterations-normalizedCost.txt', sims_acth)
np.savetxt('andersenModel_output/andersenModel-nelson-patientMean-sims-cort-5-iterations-normalizedCost.txt', sims_cort)

In [ ]:
np.savetxt('andersenModel_output/andersenModel-nelson-patientMean-initial-conditions-5-iterations-normalizedCost.txt', y0)
np.savetxt('andersenModel_output/andersenModel-nelson-patientMean-bounds-5-iterations-normalizedCost.txt', bounds)

[Back to Top](#top)

## Compute Means and Std Devations of Parameters and Output as Table <a name="paramtable"></a>

In [ ]:
# compute parameter means and standard deviations
k0_mean = np.mean(opt_pars[:,1])
k0_std = np.std(opt_pars[:,1])
k1_mean = np.mean(opt_pars[:,2])
k1_std = np.std(opt_pars[:,2])
k2_mean = np.mean(opt_pars[:,3])
k2_std = np.std(opt_pars[:,3])
w1_mean = np.mean(opt_pars[:,4])
w1_std = np.std(opt_pars[:,4])
w2_mean = np.mean(opt_pars[:,5])
w2_std = np.std(opt_pars[:,5])
w3_mean = np.mean(opt_pars[:,6])
w3_std = np.std(opt_pars[:,6])
rho_mean = np.mean(opt_pars[:,7])
rho_std = np.std(opt_pars[:,7])
psi_mean = np.mean(opt_pars[:,8])
psi_std = np.std(opt_pars[:,8])
xi_mean = np.mean(opt_pars[:,9])
xi_std = np.std(opt_pars[:,9])
alpha_mean = np.mean(opt_pars[:,10])
alpha_std = np.std(opt_pars[:,10])
gamma_mean = np.mean(opt_pars[:,11])
gamma_std = np.std(opt_pars[:,11])
c_mean = np.mean(opt_pars[:,12])
c_std = np.std(opt_pars[:,12])
c3_mean = np.mean(opt_pars[:,13])
c3_std = np.std(opt_pars[:,13])

In [ ]:
# print a table of parameter means and standard deviations
print(tabulate([["k0", "%f +- %f" % (k0_mean, k0_std)], ["k1", "%f +- %f" % (k1_mean, k1_std)], ["k2", "%f +- %f" % (k2_mean, k2_std)], ["w1", "%f +- %f" % (w1_mean, w1_std)], ["w2", "%f +- %f" % (w2_mean, w2_std)], ["w3", "%f +- %f" % (w3_mean, w3_std)], ["rho", "%f +- %f" % (rho_mean, rho_std)], ["psi", "%f +- %f" % (psi_mean, psi_std)], ["xi", "%f +- %f" % (xi_mean, xi_std)], ["alpha", "%f +- %f" % (alpha_mean, alpha_std)], ["gamma", "%f +- %f" % (gamma_mean, gamma_std)], ["c", "%f +- %f" % (c_mean, c_std)], ["c3", "%f +- %f" % (c3_mean, c3_std)]], headers = ["Parameter", "Mean +- Standard Deviation"]))

In [ ]:
# save parameter means and std devs to a file
np.savetxt('andersenModel_output/andersenModel-nelson-patientMean-param-means-stds-5-iterations-normalizedCost.txt', [k0_mean, k0_std, k1_mean, k1_std, k2_mean, k2_std, w1_mean, w1_std, w2_mean, w2_std, w3_mean, w3_std, rho_mean, rho_std, psi_mean, psi_std, xi_mean, xi_std, alpha_mean, alpha_std, gamma_mean, gamma_std, c_mean, c_std, c3_mean, c3_std])

[Back to Top](#top)

## Plots <a name="plots"></a>

In [ ]:
%matplotlib inline
matplotlib.rc('font', **{'size'   : 28})

# plot with 3 subfigures--one for CRH, one for ACTH and one for cortisol
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (24, 35))

# if running more than one iteration of optimization, uncomment plot command with np.mean and fill_between
# and comment plot command without np.mean
#
# if only one run is stored in sims variables, comment out plot commands with np.mean and fill_between

ax1.plot(optimizedSimData[:,0], sims_crh, label = "Simulated CRH Data", color = "blue")
#ax1.plot(optimizedSimData[:,0], np.mean(sims_crh, axis = 1), label = "Mean Simulated CRH", color = "blue")
#ax1.fill_between(optimizedSimData[:,0], np.mean(sims_crh, axis = 1) - np.std(sims_crh, axis = 1), np.mean(sims_crh, axis = 1) + np.std(sims_crh, axis = 1), alpha = 0.3, label = "Simulated CRH Standard Deviation")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (min)", ylabel = "CRH (micrograms/dL)", title = "CRH Concentration")

# change this line when you change which data set you are matching
ax2.plot(nelson.ACTH[:,0], nelson.ACTH[:,1], label = "Nelson ACTH Data - Patient Mean", color = "orange")

# for only 1 iteration, uncomment below and comment the following line
ax2.plot(optimizedSimData[:,0], sims_acth, label = "Simulated ACTH Data", color = "blue")
#ax2.plot(optimizedSimData[:,0], np.mean(sims_acth, axis = 1), label = "Mean Simulated ACTH", color = "blue")
#ax2.fill_between(optimizedSimData[:,0], np.mean(sims_acth, axis = 1) - np.std(sims_acth, axis = 1), np.mean(sims_acth, axis = 1) + np.std(sims_acth, axis = 1), alpha = 0.3, label = "Simulated ACTH Standard Deviation")
ax2.legend(loc = "upper left", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (min)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

# change this line when you change which data set you are matching
ax3.plot(nelson.cortisol[:,0], nelson.cortisol[:,1], label = "Nelson Cortisol Data - Patient Mean", color = "orange")

# for only 1 iteration, uncomment below and comment the following line
ax3.plot(optimizedSimData[:,0], sims_cort, label = "Simulated Cortisol Data", color = "blue")
#ax3.plot(optimizedSimData[:,0], np.mean(sims_cort, axis = 1), label = "Mean Simulated Cortisol", color = "blue")
#ax3.fill_between(optimizedSimData[:,0], np.mean(sims_cort, axis = 1) - np.std(sims_cort, axis = 1), np.mean(sims_cort, axis = 1) + np.std(sims_cort, axis = 1), alpha = 0.3, label = "Simulated Cortisol Standard Deviation")
ax3.legend(loc = "upper left", shadow = True, fancybox = True)
ax3.set(xlabel = "Time (min)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

# include the value of the cost function for this set of graphs at the bottom of the figure
txt = "cost = []"
plt.figtext(0.1, 0.09, txt, wrap = True, horizontalalignment = 'center', fontsize = 30)
#plt.savefig("andersenModel_output/andersenModel-nelson-patientMean-5-iterations-normalizedCost.png", dpi = 300)

[Back to Top](#top)

## No Optimization Run and Plot <a name="no-opt"></a>

In [ ]:
y0 = [1, nelson.ACTH[0,51], nelson.cortisol[0,51]]

In [ ]:
%%time

# run the solver with authors' published parameters
optimizedSimData = model(authors_params, y0)
    
# save CRH, cortisol and ACTH data into sims arrays
sims_cort = optimizedSimData[:,3]
sims_acth = optimizedSimData[:,2]
sims_crh = optimizedSimData[:,1]

In [ ]:
%matplotlib inline
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(optimizedSimData[:,0], sims_crh)
ax2.plot(optimizedSimData[:,0], sims_acth)
ax3.plot(optimizedSimData[:,0], sims_cort)

[Back to Top](#top)

## Reproducing Figure 6.1 from Paper <a name="reproducing"></a>

In [ ]:
# non-dimensional parameters
w1t = 4.79
w2t = 0.964
w3t = 0.251
c3t = 0.464
psi = 0.5
rho = 0.5
xi = 2
gamma = 3
alpha = 3

nd_params = [w1t, w2t, w3t, c3t, psi, rho, xi, gamma, alpha]

In [ ]:
X3 = np.linspace(0, 2, num = 250)

In [ ]:
def H(x):
    a = (w1t*w2t*w3t)**(-1)
    b = (1 + xi*((x**alpha)/(1 + x**alpha)) - psi*((x**gamma)/(c3t**gamma + x**gamma)))
    c = (1 - rho*((x**alpha)/(1 + x**alpha)))
    
    return a*b*c

In [ ]:
HX3 = H(X3)

In [ ]:
%matplotlib inline

fig, (ax1) = plt.subplots(nrows = 1, figsize = (15,5))

ax1.plot(X3, HX3, label = "H(X3)", color = "blue")
ax1.plot(X3, X3, label = "L(X3)", color = "orange")
ax1.set(title = "Figure 6.1", xlabel = "X3")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)

plt.savefig("andersenModel_output/andersenModel-figure6.1-reproduction.png", dpi = 300)

[Back to Top](#top)

## Run from Opt_pars File <a name="run-optpars"></a>

In [ ]:
print(nelson.ACTH[0,2], nelson.cortisol[0,2])

In [ ]:
y0 = [8.0, 33.6, 7.4]

t_start = -0.01
t_end = 140.01
t_step = 0.01

In [ ]:
loaded_params = np.loadtxt("andersenModel_output/andersenModel-nelson-patient1-opt-pars-5-iterations.txt")

In [ ]:
sims_cort = np.zeros((14002, len(loaded_params[:,0])))
sims_acth = np.zeros((14002, len(loaded_params[:,0])))
sims_crh = np.zeros((14002, len(loaded_params[:,0])))

In [ ]:
%%time

for index, value in enumerate(loaded_params):
    optimizedSimData = model(value[1:], y0)

    sims_cort[:,index] = optimizedSimData[:,3]
    sims_acth[:,index] = optimizedSimData[:,2]
    sims_crh[:,index] = optimizedSimData[:,1]

In [ ]:
print(np.sort(loaded_params[:,0]))

In [ ]:
%matplotlib inline

# plot with 3 subfigures--one for CRH, one for ACTH and one for cortisol
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15, 15))

# if running more than one iteration of optimization, uncomment plot command with np.mean and fill_between
# and comment plot command without np.mean
#
# if only one run is stored in sims variables, comment out plot commands with np.mean and fill_between
#ax1.plot(optimizedSimData[:,0], sims_crh, label = "Simulated CRH Data", color = "blue")
ax1.plot(optimizedSimData[:,0], np.mean(sims_crh, axis = 1), label = "Mean Simulated CRH", color = "blue")
ax1.fill_between(optimizedSimData[:,0], np.mean(sims_crh, axis = 1) - np.std(sims_crh, axis = 1), np.mean(sims_crh, axis = 1) + np.std(sims_crh, axis = 1), alpha = 0.3, label = "Simulated CRH Standard Deviation")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (min)", ylabel = "CRH (micrograms/dL)", title = "CRH Concentration")

ax2.plot(nelson.ACTH[:,0], nelson.ACTH[:,2], label = "Nelson ACTH Data - Patient 1", color = "orange")
#ax2.plot(optimizedSimData[:,0], sims_acth, label = "Simulated ACTH Data", color = "blue")
ax2.plot(optimizedSimData[:,0], np.mean(sims_acth, axis = 1), label = "Mean Simulated ACTH", color = "blue")
ax2.fill_between(optimizedSimData[:,0], np.mean(sims_acth, axis = 1) - np.std(sims_acth, axis = 1), np.mean(sims_acth, axis = 1) + np.std(sims_acth, axis = 1), alpha = 0.3, label = "Simulated ACTH Standard Deviation")
ax2.legend(loc = "upper left", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (min)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax3.plot(nelson.cortisol[:,0], nelson.cortisol[:,2], label = "Nelson Cortisol Data - Patient 1", color = "orange")
#ax3.plot(optimizedSimData[:,0], sims_cort, label = "Simulated Cortisol Data", color = "blue")
ax3.plot(optimizedSimData[:,0], np.mean(sims_cort, axis = 1), label = "Mean Simulated Cortisol", color = "blue")
ax3.fill_between(optimizedSimData[:,0], np.mean(sims_cort, axis = 1) - np.std(sims_cort, axis = 1), np.mean(sims_cort, axis = 1) + np.std(sims_cort, axis = 1), alpha = 0.3, label = "Simulated Cortisol Standard Deviation")
ax3.legend(loc = "upper left", shadow = True, fancybox = True)
ax3.set(xlabel = "Time (min)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

# include the value of the cost function for this set of graphs at the bottom of the figure
txt = "cost = [236.67405379]"
plt.figtext(0.1, 0.09, txt, wrap = True, horizontalalignment = 'center', fontsize = 12)
plt.savefig("andersenModel_output/andersenModel-nelson-patient1-5-iterations.png", dpi = 300)

[Back to Top](#top)

## Dependencies <a name="dependencies"></a>

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversions

[Back to Top](#top)